In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.tokenize import word_tokenize
import nltk

# Assicurati di avere i token di NLTK per il tokenization
nltk.download('punkt')



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ANGELAANGUILANO\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:

# Caricamento del testo (usa un testo di esempio o un libro)
with open('testo_autore.txt', 'r', encoding='utf-8') as file:
    text = file.read().lower()

# Tokenizzazione del testo
tokens = word_tokenize(text)

# Creazione del vocabolario unico (mappa le parole in indici)
vocab = sorted(set(tokens))
vocab_size = len(vocab)
word_to_int = {word: i for i, word in enumerate(vocab)}
int_to_word = {i: word for i, word in enumerate(vocab)}

# Creazione delle sequenze di input (X) e target (y)
seq_length = 40  # Numero di parole in ciascuna sequenza
sequences = []

for i in range(seq_length, len(tokens)):
    seq = tokens[i-seq_length:i]  # Sequenza di 40 parole
    label = tokens[i]  # Parola successiva
    sequences.append([word_to_int[word] for word in seq] + [word_to_int[label]])

# Creazione degli input (X) e degli output (y)
sequences = np.array(sequences)
X, y = sequences[:, :-1], sequences[:, -1]


In [3]:

# Creazione del modello GRU
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=seq_length-1))
model.add(GRU(256, return_sequences=True))  # GRU invece di LSTM
model.add(Dropout(0.2))
model.add(GRU(256))  # GRU classico
model.add(Dense(vocab_size, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Preparazione di y (one-hot encoding)
y = to_categorical(y, num_classes=vocab_size)

# Addestramento del modello
model.fit(X, y, epochs=50, batch_size=128)


Epoch 1/50


C:\Users\ANGELAANGUILANO\simple-environment\.venv\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


11/11 ━━━━━━━━━━━━━━━━━━━━ 3s 152ms/step - accuracy: 0.0402 - loss: 6.0413
Epoch 2/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 151ms/step - accuracy: 0.0320 - loss: 5.3577
Epoch 3/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 141ms/step - accuracy: 0.0492 - loss: 5.2050
Epoch 4/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 136ms/step - accuracy: 0.0604 - loss: 5.1383
Epoch 5/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 136ms/step - accuracy: 0.0797 - loss: 5.0794
Epoch 6/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 132ms/step - accuracy: 0.0879 - loss: 4.9950
Epoch 7/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 129ms/step - accuracy: 0.0991 - loss: 4.8820
Epoch 8/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 134ms/step - accuracy: 0.1073 - loss: 4.7813
Epoch 9/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 133ms/step - accuracy: 0.1311 - loss: 4.6162
Epoch 10/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 134ms/step - accuracy: 0.1542 - loss: 4.4541
Epoch 11/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 130ms/step - accuracy: 0.1833 - loss: 4.2799
Epoch 12/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 130ms/step - accuracy

In [4]:

# Funzione per generare del testo
def generate_text(model, int_to_word, word_to_int, seed_text, seq_length, num_words):
    # Inizializza il testo con il seed_text
    result = seed_text
    input_text = seed_text.lower().split()
    
    # Genera 'num_words' parole
    for _ in range(num_words):
        # Converte il testo in sequenza numerica
        encoded = [word_to_int[word] for word in input_text if word in word_to_int]
        
        # Padding per la sequenza (poiché la sequenza di input deve avere una lunghezza fissa)
        encoded = pad_sequences([encoded], maxlen=seq_length-1, padding='pre')
        
        # Predice la parola successiva
        pred_probs = model.predict(encoded, verbose=0)
        predicted_idx = np.argmax(pred_probs)
        
        # Converte l'indice predetto in parola
        predicted_word = int_to_word[predicted_idx]
        
        # Aggiungi la parola predetta al testo
        result += ' ' + predicted_word
        
        # Aggiungi la parola predetta alla sequenza di input per la prossima previsione
        input_text.append(predicted_word)
        input_text = input_text[1:]
    
    return result



In [5]:
# Genera un testo nello stile dell'autore
seed_text = "era una notte buia"  # Testo iniziale
generated_text = generate_text(model, int_to_word, word_to_int, seed_text, seq_length, 100)
print(generated_text)

era una notte buia che pronuncerà sta cercando di cercando di tua scuola . se non è lui , che ha appena deliberato tonnellate ora ora che rimpinza di volantini deliberato tonnellate di ora che rimpinza di volantini deliberato tonnellate di ora che rimpinza di volantini deliberato tonnellate di ora che rimpinza di volantini deliberato tonnellate di ora che rimpinza di volantini deliberato tonnellate di ora che rimpinza di volantini deliberato tonnellate di ora che rimpinza di volantini deliberato tonnellate di ora che rimpinza di volantini deliberato tonnellate di ora che rimpinza di volantini deliberato tonnellate di ora che rimpinza di volantini deliberato tonnellate
